<a href="https://colab.research.google.com/github/BooEiEi/Master-Project/blob/main/Hyperparameter_XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF'
path_DF2 = '/content/drive/My Drive/master project/data/DF2'

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error

In [ ]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF2, 'df_GOOG.csv'))

In [ ]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])
for lag in [1, 3, 5, 7]:
    df_AAPL[f'lag_{lag}'] = df_AAPL['Stockprice_AAPL'].shift(lag)

# ลบค่าที่เป็น NaN จากการสร้าง lag features
df_AAPL.dropna(inplace=True)

In [ ]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])
for lag in [1, 3, 5, 7]:
    df_MSFT[f'lag_{lag}'] = df_MSFT['Stockprice_MSFT'].shift(lag)

# ลบค่าที่เป็น NaN จากการสร้าง lag features
df_MSFT.dropna(inplace=True)

In [ ]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])
for lag in [1, 3, 5, 7]:
    df_NVDA[f'lag_{lag}'] = df_NVDA['Stockprice_NVDA'].shift(lag)

# ลบค่าที่เป็น NaN จากการสร้าง lag features
df_NVDA.dropna(inplace=True)

In [ ]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])

In [ ]:
train_size = int(len(df_META) * 0.7)  # 70% ของ 753
test_size = len(df_META) - train_size  # 30% ที่เหลือ

# META

## Train Test

In [ ]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_META = df_META.iloc[:train_size]  # ข้อมูล train
test_META = df_META.iloc[train_size:]  # ข้อมูล test

### Train Varidation

In [ ]:
X_META = train_META.drop(columns=["Stockprice_META","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_META = train_META["Stockprice_META"]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

X_META = scaler.fit_transform(X_META)

In [ ]:
model = XGBRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_META , y_META)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_META)
mse = mean_squared_error(y_META, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

Best Hyperparameters: {'alpha': 0.1, 'colsample_bytree': 0.5, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 600, 'reg_lambda': 0.1}
Test Set Mean Squared Error: 0.2529


#AAPL

In [ ]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_AAPL = df_AAPL.iloc[:train_size]  # ข้อมูล train
test_AAPL = df_AAPL.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_AAPL = train_AAPL.drop(columns=["Stockprice_AAPL","date", "Interest", "US_Treasury_Yield", "RETAIL_SALES", "US_Dollar_Index"])
y_AAPL = train_AAPL["Stockprice_AAPL"]

In [ ]:
X_AAPL = scaler.fit_transform(X_AAPL)

In [ ]:
model = XGBRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_AAPL , y_AAPL)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_AAPL)
mse = mean_squared_error(y_AAPL, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

Best Hyperparameters: {'alpha': 0, 'colsample_bytree': 0.1, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 600, 'reg_lambda': 0.1}
Test Set Mean Squared Error: 1.5950


#MSFT

In [ ]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_MSFT = df_MSFT.iloc[:train_size]  # ข้อมูล train
test_MSFT = df_MSFT.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_MSFT = train_MSFT.drop(columns=["Stockprice_MSFT","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_MSFT = train_MSFT["Stockprice_MSFT"]

In [ ]:
X_MSFT = scaler.fit_transform(X_MSFT)

In [ ]:
model = XGBRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_MSFT , y_MSFT)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_MSFT)
mse = mean_squared_error(y_MSFT, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

Best Hyperparameters: {'alpha': 0, 'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 600, 'reg_lambda': 0}
Test Set Mean Squared Error: 0.0122


#NVDA

In [ ]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_NVDA = train_NVDA[["sentiment_NVDA", "Interest", "CPI", "BTC", "S&P500", "lag_1", "lag_3", "lag_5", "lag_7"]]
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [ ]:
model = XGBRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_NVDA , y_NVDA)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_NVDA)
mse = mean_squared_error(y_NVDA, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

Best Hyperparameters: {'alpha': 10, 'colsample_bytree': 0.1, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 600, 'reg_lambda': 10}
Test Set Mean Squared Error: 246.4914


#Add variable

In [ ]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_NVDA = train_NVDA[["sentiment_NVDA", "Interest", "CPI", "BTC", "S&P500", "lag_1", "lag_3", "lag_5", "lag_7"]]
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [ ]:
X_NVDA = scaler.fit_transform(X_NVDA)

In [ ]:
model = XGBRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_NVDA , y_NVDA)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_NVDA)
mse = mean_squared_error(y_NVDA, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

Best Hyperparameters: {'alpha': 1, 'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 200, 'reg_lambda': 10}
Test Set Mean Squared Error: 9.7780


#GOOG

In [ ]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_GOOG = df_GOOG.iloc[:train_size]  # ข้อมูล train
test_GOOG = df_GOOG.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_GOOG = train_GOOG.drop(columns=["Stockprice_GOOGL","date", "Interest", "RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_GOOG = train_GOOG["Stockprice_GOOGL"]

In [ ]:
X_GOOG = scaler.fit_transform(X_GOOG)

In [ ]:
model = XGBRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'alpha': [0, 0.1, 1, 10],
    'reg_lambda' : [0, 0.1, 1, 10],
    'n_estimators' : [100, 200, 400, 600],
    'min_child_weight' : [ 1, 3, 5],
    'colsample_bytree' : [0.1, 0.3, 0.5, 0.7]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_GOOG , y_GOOG)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_GOOG)
mse = mean_squared_error(y_GOOG, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

Best Hyperparameters: {'alpha': 0, 'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'reg_lambda': 0}
Test Set Mean Squared Error: 1.1096
